In [1]:
!pip install pydot
!pip install --upgrade mxnet-cu90 --pre
!pip install --upgrade keras-mxnet --pre

mxnet-model-server 0.3 requires fasteners, which is not installed.
mxnet-model-server 0.3 requires importlib2, which is not installed.
mxnet-model-server 0.3 requires mxnet>=1.1, which is not installed.
mxnet-model-server 0.3 requires onnx-mxnet>=0.4.2, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
keras 2.2.2 has requirement keras-applications==1.0.4, but you'll have keras-applications 1.0.2 which is incompatible.
keras 2.2.2 has requirement keras-preprocessing==1.0.2, but you'll have keras-preprocessing 1.0.1 which is incompatible.
You are using pip version 10.0.1, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Requirement already up-to-date: mxnet-cu90 in /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages (1.3.0b20180809)
Requirement not upgraded as not directly required: requests<2.19.0,>=2.18.4 in /home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages

In [2]:
import mxnet as mx
import keras

Using MXNet backend


In [3]:
print("mxnet_version: {}\nkeras_version: {}\nkeras checkpointing class: {}".format(mx.__version__, 
                                                                                   keras.__version__,
                                                                                   keras.callbacks.MXNetModelCheckpoint))


mxnet_version: 1.3.0
keras_version: 2.2.0
keras checkpointing class: <class 'keras.callbacks.MXNetModelCheckpoint'>


# A first example
In the following cells we implement CIFAR 10 CNN

In [4]:
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPooling2D, Conv2D

# Parameters and Environment Variables

In [5]:
batch_size = 32
num_classes = 10
epochs = 10

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')
print(type(y_test))
print(y_test)

x_train shape: (50000, 32, 32, 3)
(50000, 32, 32, 3) train samples
(10000, 32, 32, 3) test samples
<class 'numpy.ndarray'>
[[3]
 [8]
 [8]
 ..., 
 [5]
 [1]
 [7]]


Converting the lable from numerical to categorical.

In [7]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print('x_train shape:', x_train.shape)
print(x_train.shape, 'train samples')
print(x_test.shape, 'test samples')
y_test

x_train shape: (50000, 32, 32, 3)
(50000, 32, 32, 3) train samples
(10000, 32, 32, 3) test samples


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  0.,  0.]], dtype=float32)

We can still observe array types are numpy and not mx.nd.array

# Developing a Sequential Model

In [8]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/keras/backend/mxnet_backend.py:89: UserWarning: MXNet Backend performs best with `channels_first` format. Using `channels_last` will significantly reduce performance due to the Transpose operations. For performance improvement, please use this API`keras.utils.to_channels_first(x_input)`to transform `channels_last` data to `channels_first` format and also please change the `image_data_format` in `keras.json` to `channels_first`.Note: `x_input` is a Numpy tensor or a list of Numpy tensorRefer to: https://github.com/awslabs/keras-apache-mxnet/tree/master/docs/mxnet_backend/performance_guide.md
  train_symbol = func(*args, **kwargs)
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/keras/backend/mxnet_backend.py:92: UserWarning: MXNet Backend performs best with `channels_first` format. Using `channels_last` will significantly reduce performance due to the Transpose operations. For performance improvement,

# Defining Optimizer

In [9]:
opt = keras.optimizers.sgd(lr=.001, momentum=.9)

# Compiling the model

In [10]:
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy', 'mse'])

In [11]:
from keras.utils import plot_model
plot_model(model, to_file='./images/model.png')

![](./images/model.png)

# preprocessing data

In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train[0, 0]

array([[  59.,   62.,   63.],
       [  43.,   46.,   45.],
       [  50.,   48.,   43.],
       [  68.,   54.,   42.],
       [  98.,   73.,   52.],
       [ 119.,   91.,   63.],
       [ 139.,  107.,   75.],
       [ 145.,  110.,   80.],
       [ 149.,  117.,   89.],
       [ 149.,  120.,   93.],
       [ 131.,  103.,   77.],
       [ 125.,   99.,   76.],
       [ 142.,  115.,   91.],
       [ 144.,  112.,   86.],
       [ 137.,  105.,   79.],
       [ 129.,   97.,   71.],
       [ 137.,  106.,   79.],
       [ 134.,  106.,   76.],
       [ 124.,   97.,   64.],
       [ 139.,  113.,   78.],
       [ 139.,  112.,   75.],
       [ 133.,  105.,   69.],
       [ 136.,  105.,   74.],
       [ 139.,  108.,   77.],
       [ 152.,  120.,   89.],
       [ 163.,  131.,  100.],
       [ 168.,  136.,  108.],
       [ 159.,  129.,  102.],
       [ 158.,  130.,  104.],
       [ 158.,  132.,  108.],
       [ 152.,  125.,  102.],
       [ 148.,  124.,  103.]], dtype=float32)

# Normalizing the data

In [13]:
x_train /= 255
x_test /= 255
x_train[0][0]

array([[ 0.23137255,  0.24313726,  0.24705882],
       [ 0.16862746,  0.18039216,  0.17647059],
       [ 0.19607843,  0.1882353 ,  0.16862746],
       [ 0.26666668,  0.21176471,  0.16470589],
       [ 0.38431373,  0.28627452,  0.20392157],
       [ 0.46666667,  0.35686275,  0.24705882],
       [ 0.54509807,  0.41960785,  0.29411766],
       [ 0.56862748,  0.43137255,  0.3137255 ],
       [ 0.58431375,  0.45882353,  0.34901962],
       [ 0.58431375,  0.47058824,  0.36470589],
       [ 0.51372552,  0.40392157,  0.3019608 ],
       [ 0.49019608,  0.3882353 ,  0.29803923],
       [ 0.55686277,  0.4509804 ,  0.35686275],
       [ 0.56470591,  0.43921569,  0.33725491],
       [ 0.53725493,  0.41176471,  0.30980393],
       [ 0.50588238,  0.38039216,  0.27843139],
       [ 0.53725493,  0.41568628,  0.30980393],
       [ 0.52549022,  0.41568628,  0.29803923],
       [ 0.48627451,  0.38039216,  0.25098041],
       [ 0.54509807,  0.44313726,  0.30588236],
       [ 0.54509807,  0.43921569,  0.294

# Training the Model

In [14]:
model.fit(x=x_train, 
          y=y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
  992/50000 [..............................] - ETA: 1:57 - loss: 2.3153 - acc: 0.0978 - mean_squared_error: 0.0903

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:408: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  force_init=force_init)


50000/50000 [==============================] - 8s 169us/step - loss: 2.0976 - acc: 0.2193 - mean_squared_error: 0.0855 - val_loss: 1.8790 - val_acc: 0.3532 - val_mean_squared_error: 0.0799
Epoch 2/10
50000/50000 [==============================] - 6s 122us/step - loss: 1.7726 - acc: 0.3533 - mean_squared_error: 0.0769 - val_loss: 1.6071 - val_acc: 0.4245 - val_mean_squared_error: 0.0717
Epoch 3/10
50000/50000 [==============================] - 6s 122us/step - loss: 1.5750 - acc: 0.4248 - mean_squared_error: 0.0707 - val_loss: 1.4110 - val_acc: 0.4928 - val_mean_squared_error: 0.0648
Epoch 4/10
50000/50000 [==============================] - 6s 120us/step - loss: 1.4504 - acc: 0.4747 - mean_squared_error: 0.0662 - val_loss: 1.3209 - val_acc: 0.5298 - val_mean_squared_error: 0.0612
Epoch 5/10
50000/50000 [==============================] - 6s 119us/step - loss: 1.3656 - acc: 0.5078 - mean_squared_error: 0.0627 - val_loss: 1.2418 - val_acc: 0.5540 - val_mean_squared_error: 0.0579
Epoch 6/10


![](./images/singlegpu.png)

# Checkpointing

In [15]:
from keras.callbacks import MXNetModelCheckpoint
from keras.callbacks import ModelCheckpoint

In [16]:
chekpoint_callback = MXNetModelCheckpoint(prefix="cifar10_keras_mxnet", monitor='val_loss', save_best_only=True)
h5checkpoint = ModelCheckpoint('./models/cifar_keras_keras.h5', monitor='val_loss', save_best_only=True)

In [17]:
model.fit(x=x_train, 
          y=y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          shuffle=True, callbacks=[chekpoint_callback, h5checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 6s 120us/step - loss: 1.0398 - acc: 0.6328 - mean_squared_error: 0.0490 - val_loss: 0.9461 - val_acc: 0.6717 - val_mean_squared_error: 0.0448
MXNet Backend: Successfully exported the model as MXNet model!
MXNet symbol file -  cifar10_keras_mxnet-symbol.json
MXNet params file -  cifar10_keras_mxnet-0000.params


Model input data_names and data_shapes are: 
data_names :  ['/conv2d_1_input1']
data_shapes :  [DataDesc[/conv2d_1_input1,(32, 32, 32, 3),float32,NCHW]]


Note: In the above data_shapes, the first dimension represent the batch_size used for model training. 
You can change the batch_size for binding the module based on your inference batch_size.
Epoch 2/10
50000/50000 [==============================] - 6s 119us/step - loss: 1.0008 - acc: 0.6457 - mean_squared_error: 0.0473 - val_loss: 0.9262 - val_acc: 0.6735 - val_mean_squared_error: 0.0441
MXNet Backend: Successfully expo

# Multi-GPU

In [18]:
num_gpus = mx.context.num_gpus()
model = keras.utils.multi_gpu_model(model=model, gpus=num_gpus)
model.compile(loss='categorical_crossentropy',optimizer=opt, metrics=['accuracy', 'mse'])

plot_model(model, to_file='./models/model1.png')

![](./images/model1.png)

In [19]:
model.fit(x=x_train, 
          y=y_train, 
          batch_size=batch_size, 
          epochs=epochs, 
          validation_data=(x_test, y_test), 
          shuffle=True, callbacks=[chekpoint_callback])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10


/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/module/bucketing_module.py:408: UserWarning: Optimizer created manually outside Module but rescale_grad is not normalized to 1.0/batch_size/num_workers (1.0 vs. 0.03125). Is this intended?
  force_init=force_init)


50000/50000 [==============================] - 19s 377us/step - loss: 0.8751 - acc: 0.6939 - mean_squared_error: 0.0416 - val_loss: 0.7935 - val_acc: 0.7188 - val_mean_squared_error: 0.0383
Epoch 2/10
50000/50000 [==============================] - 12s 236us/step - loss: 0.8140 - acc: 0.7118 - mean_squared_error: 0.0391 - val_loss: 0.8065 - val_acc: 0.7250 - val_mean_squared_error: 0.0380
Epoch 3/10
50000/50000 [==============================] - 12s 235us/step - loss: 0.7644 - acc: 0.7316 - mean_squared_error: 0.0369 - val_loss: 0.7375 - val_acc: 0.7441 - val_mean_squared_error: 0.0353
Epoch 4/10
50000/50000 [==============================] - 12s 235us/step - loss: 0.7272 - acc: 0.7443 - mean_squared_error: 0.0353 - val_loss: 0.6957 - val_acc: 0.7594 - val_mean_squared_error: 0.0335
MXNet Backend: Successfully exported the model as MXNet model!
MXNet symbol file -  cifar10_keras_mxnet-symbol.json
MXNet params file -  cifar10_keras_mxnet-0000.params


Model input data_names and data_shap

![](./images/multigpu.png)

In [23]:
sym, arg_params, aux_params = mx.model.load_checkpoint('./models/cifar10_keras_mxnet', 0)
print(x_train.shape)

(50000, 32, 32, 3)



# Still have to do inference code... nothing major